## Geo Data

#### In this df I am going to link the theft information with geographic information using the Google Maps API.

In [9]:
#pip install googlemaps 

import pandas as pd
import googlemaps
from googlemaps.exceptions import ApiError

In [44]:
# Read the csv file with theft locations 

df_bikes = pd.read_csv('df_final.csv')
df_bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55004 entries, 0 to 55003
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Record_date              55004 non-null  object
 1   Stealing_date            55004 non-null  object
 2   Stealing_hour            55004 non-null  int64 
 3   Report_stealing_date     55004 non-null  object
 4   Report_stealing_hour     55004 non-null  int64 
 5   Berlin_code_area         55004 non-null  int64 
 6   Bike_value               55004 non-null  int64 
 7   Unsuccesful_attempt      55004 non-null  object
 8   Bike_type                55004 non-null  object
 9   Crime_type               55004 non-null  object
 10  Crime_reason             55004 non-null  object
 11  Administrative_district  55004 non-null  object
 12  District                 55004 non-null  object
 13  Quarter                  55004 non-null  object
 14  Street_name              55004 non-nul

#### Geocoding API documentation: 
https://developers.google.com/maps/documentation/geocoding?hl=en

Defining a function to obtain the coordinates (latitude and longitude) of the streets where the robberies occurred.

In [3]:
def get_latitude_longitude(api_key, address):
    gmaps = googlemaps.Client(key=api_key)
    try:
        geocode_result = gmaps.geocode(address)
        if geocode_result:
            latitude = geocode_result[0]['geometry']['location']['lat']
            longitude = geocode_result[0]['geometry']['location']['lng']
            return latitude, longitude
        else:
            return None, None
    except ApiError as e:
        print(f"Error occurred: {e}")
        return None, None

In [ ]:
#api_key = 

latitudes = []
longitudes = []

for street_name in df_bikes['Street_name']:
    address = street_name
    latitude, longitude = get_latitude_longitude(api_key, address)
    latitudes.append(latitude)
    longitudes.append(longitude)

# Add 'latitude' and 'longitude' columns to the DataFrame
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [40]:
df_streets = pd.read_excel('ber_streets_fix.xlsx')
df_streets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Borough            447 non-null    object 
 1   City               643 non-null    object 
 2   Street_name        643 non-null    object 
 3   Latitude           643 non-null    float64
 4   Longitude          643 non-null    float64
 5   Number of Records  643 non-null    int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 30.3+ KB


In [13]:
df_streets.head()

,Borough,City,Street_name,Latitude,Longitude,Number of Records
0,Spandau,Berlin,Ackerstraße,52.547137,13.197814,1
1,Spandau,Berlin,Adamstraße,52.521004,13.192730,1
2,Marzahn-Hellersdorf,Berlin,Adele-Sandrock-Straße,52.534720,13.619069,1
3,Treptow-Köpenick,Berlin,Adlershof Ost,52.444606,13.555729,1
4,Treptow-Köpenick,Berlin,Adlershof West,52.432572,13.521053,1


In [41]:
merged_df = pd.merge(df_bikes,df_streets, on='Street_name', how='left')

In [43]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55004 entries, 0 to 55003
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              55004 non-null  object 
 1   Stealing_date            55004 non-null  object 
 2   Stealing_hour            55004 non-null  int64  
 3   Report_stealing_date     55004 non-null  object 
 4   Report_stealing_hour     55004 non-null  int64  
 5   Berlin_code_area         55004 non-null  int64  
 6   Bike_value               55004 non-null  int64  
 7   Unsuccesful_attempt      55004 non-null  object 
 8   Bike_type                55004 non-null  object 
 9   Crime_type               55004 non-null  object 
 10  Crime_reason             55004 non-null  object 
 11  Administrative_district  55004 non-null  object 
 12  District                 55004 non-null  object 
 13  Quarter                  55004 non-null  object 
 14  Street_name           

In [46]:
merged_df.drop(columns='Borough', inplace=True)

In [47]:
filtered_df = merged_df[merged_df['Latitude'].isnull()]

In [48]:
filtered_df

,Record_date,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name,City,Latitude,Longitude,Number of Records
224,2022-11-27,2022-11-27,2,2022-11-27,6,2300316,1594,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,NaN,NaN,NaN,NaN
227,2022-11-27,2022-11-26,19,2022-11-26,19,2300316,950,No,Men,Theft,Theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,NaN,NaN,NaN,NaN
245,2022-11-27,2022-11-27,9,2022-11-27,14,2300316,450,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,NaN,NaN,NaN,NaN
764,2022-11-17,2022-11-09,22,2022-11-10,11,2300316,500,No,Women,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,NaN,NaN,NaN,NaN
893,2022-11-16,2022-11-15,19,2022-11-16,8,2300316,440,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53798,2022-12-11,2022-12-09,21,2022-12-10,2,2300316,1200,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,NaN,NaN,NaN,NaN
53915,2022-12-08,2022-10-16,14,2022-10-16,16,4300412,200,No,Women,Theft,Aggravated theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Schloss Charlottenburg,Schlossgarten,NaN,NaN,NaN,NaN
54252,2022-11-29,2022-10-13,13,2022-10-13,22,4300412,400,No,Men,Theft,Theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Schloss Charlottenburg,Schlossgarten,NaN,NaN,NaN,NaN
54417,2022-10-26,2022-10-25,10,2022-10-25,15,2300316,479,No,Mountain,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,NaN,NaN,NaN,NaN


In [21]:
filtered_df.to_csv('filtered.csv', index=False)

In [49]:
cleaned_df = merged_df.dropna()
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54769 entries, 0 to 55003
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              54769 non-null  object 
 1   Stealing_date            54769 non-null  object 
 2   Stealing_hour            54769 non-null  int64  
 3   Report_stealing_date     54769 non-null  object 
 4   Report_stealing_hour     54769 non-null  int64  
 5   Berlin_code_area         54769 non-null  int64  
 6   Bike_value               54769 non-null  int64  
 7   Unsuccesful_attempt      54769 non-null  object 
 8   Bike_type                54769 non-null  object 
 9   Crime_type               54769 non-null  object 
 10  Crime_reason             54769 non-null  object 
 11  Administrative_district  54769 non-null  object 
 12  District                 54769 non-null  object 
 13  Quarter                  54769 non-null  object 
 14  Street_name           

In [50]:
missing_fixed = pd.read_excel('filtered_fixed.xlsx')

In [51]:
missing_fixed

,Record_date,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name,Latitude,Longitude,Number of Records
0,2022-11-27,2022-11-27,2,2022-11-27,6,2300316,1594,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,NaN
1,2022-11-27,2022-11-26,19,2022-11-26,19,2300316,950,No,Men,Theft,Theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,NaN
2,2022-11-27,2022-11-27,9,2022-11-27,14,2300316,450,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,NaN
3,2022-11-17,2022-11-09,22,2022-11-10,11,2300316,500,No,Women,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,NaN
4,2022-11-16,2022-11-15,19,2022-11-16,8,2300316,440,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,2022-12-11,2022-12-09,21,2022-12-10,2,2300316,1200,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,NaN
231,2022-12-08,2022-10-16,14,2022-10-16,16,4300412,200,No,Women,Theft,Aggravated theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Schloss Charlottenburg,Schlossgarten,52.537872,13.324342,NaN
232,2022-11-29,2022-10-13,13,2022-10-13,22,4300412,400,No,Men,Theft,Theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Schloss Charlottenburg,Schlossgarten,52.537872,13.324342,NaN
233,2022-10-26,2022-10-25,10,2022-10-25,15,2300316,479,No,Mountain,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,NaN


In [52]:
df_final = pd.concat([cleaned_df, missing_fixed], ignore_index=True)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55004 entries, 0 to 55003
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              55004 non-null  object 
 1   Stealing_date            55004 non-null  object 
 2   Stealing_hour            55004 non-null  int64  
 3   Report_stealing_date     55004 non-null  object 
 4   Report_stealing_hour     55004 non-null  int64  
 5   Berlin_code_area         55004 non-null  int64  
 6   Bike_value               55004 non-null  int64  
 7   Unsuccesful_attempt      55004 non-null  object 
 8   Bike_type                55004 non-null  object 
 9   Crime_type               55004 non-null  object 
 10  Crime_reason             55004 non-null  object 
 11  Administrative_district  55004 non-null  object 
 12  District                 55004 non-null  object 
 13  Quarter                  55004 non-null  object 
 14  Street_name           

In [53]:
df_final['Number of Records'] = df_final['Number of Records'].fillna(1)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55004 entries, 0 to 55003
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              55004 non-null  object 
 1   Stealing_date            55004 non-null  object 
 2   Stealing_hour            55004 non-null  int64  
 3   Report_stealing_date     55004 non-null  object 
 4   Report_stealing_hour     55004 non-null  int64  
 5   Berlin_code_area         55004 non-null  int64  
 6   Bike_value               55004 non-null  int64  
 7   Unsuccesful_attempt      55004 non-null  object 
 8   Bike_type                55004 non-null  object 
 9   Crime_type               55004 non-null  object 
 10  Crime_reason             55004 non-null  object 
 11  Administrative_district  55004 non-null  object 
 12  District                 55004 non-null  object 
 13  Quarter                  55004 non-null  object 
 14  Street_name           

In [54]:
df_final.drop(columns='City', inplace=True)

In [60]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55004 entries, 0 to 55003
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              55004 non-null  object 
 1   Stealing_date            55004 non-null  object 
 2   Stealing_hour            55004 non-null  int64  
 3   Report_stealing_date     55004 non-null  object 
 4   Report_stealing_hour     55004 non-null  int64  
 5   Berlin_code_area         55004 non-null  int64  
 6   Bike_value               55004 non-null  int64  
 7   Unsuccesful_attempt      55004 non-null  object 
 8   Bike_type                55004 non-null  object 
 9   Crime_type               55004 non-null  object 
 10  Crime_reason             55004 non-null  object 
 11  Administrative_district  55004 non-null  object 
 12  District                 55004 non-null  object 
 13  Quarter                  55004 non-null  object 
 14  Street_name           

In [56]:
df_final.to_csv('df_final.csv', index=False)

In [57]:
df_final.to_excel('df_tableau.xlsx',index=False)

In [29]:
east_west_df = pd.read_excel('east_west.xlsx')

In [58]:
east_west_df

,Administrative District,District,East/West
0,Charlottenburg-Wilmersdorf,Charlottenburg Nord,West
1,Charlottenburg-Wilmersdorf,Charlottenburg West,West
2,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,West
3,Charlottenburg-Wilmersdorf,Wilmersdorf Süd,West
4,Charlottenburg-Wilmersdorf,Wilmersdorf Zentrum,West
5,Friedrichshain-Kreuzberg,Friedrichshain Ost,East
6,Friedrichshain-Kreuzberg,Friedrichshain West,East
7,Friedrichshain-Kreuzberg,Kreuzberg Nord,West
8,Friedrichshain-Kreuzberg,Kreuzberg Ost,West
9,Friedrichshain-Kreuzberg,Kreuzberg Süd,West


In [59]:
east_west_df.rename(columns={'Neighborhood': 'District'}, inplace=True)

In [32]:
east_west_df

,Administrative District,District,East/West
0,Charlottenburg-Wilmersdorf,Charlottenburg Nord,West
1,Charlottenburg-Wilmersdorf,Charlottenburg West,West
2,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,West
3,Charlottenburg-Wilmersdorf,Wilmersdorf Süd,West
4,Charlottenburg-Wilmersdorf,Wilmersdorf Zentrum,West
5,Friedrichshain-Kreuzberg,Friedrichshain Ost,East
6,Friedrichshain-Kreuzberg,Friedrichshain West,East
7,Friedrichshain-Kreuzberg,Kreuzberg Nord,West
8,Friedrichshain-Kreuzberg,Kreuzberg Ost,West
9,Friedrichshain-Kreuzberg,Kreuzberg Süd,West


In [61]:
df_west_east = pd.merge(df_final,east_west_df, on='District', how='left')

In [63]:
df_west_east.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55004 entries, 0 to 55003
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              55004 non-null  object 
 1   Stealing_date            55004 non-null  object 
 2   Stealing_hour            55004 non-null  int64  
 3   Report_stealing_date     55004 non-null  object 
 4   Report_stealing_hour     55004 non-null  int64  
 5   Berlin_code_area         55004 non-null  int64  
 6   Bike_value               55004 non-null  int64  
 7   Unsuccesful_attempt      55004 non-null  object 
 8   Bike_type                55004 non-null  object 
 9   Crime_type               55004 non-null  object 
 10  Crime_reason             55004 non-null  object 
 11  Administrative_district  55004 non-null  object 
 12  District                 55004 non-null  object 
 13  Quarter                  55004 non-null  object 
 14  Street_name           

In [102]:
df_west_east['Administrative_district'].value_counts()

Mitte                         8330
Pankow                        8029
Friedrichshain-Kreuzberg      7737
Charlottenburg-Wilmersdorf    5692
Tempelhof-Schöneberg          4737
Neukölln                      4178
Treptow-Köpenick              4176
Steglitz-Zehlendorf           3512
Lichtenberg                   3188
Reinickendorf                 2159
Marzahn-Hellersdorf           1729
Spandau                       1537
Name: Administrative_district, dtype: int64

In [105]:
spandau_df = df_west_east[df_west_east['Administrative_district'] == 'Spandau']
spandau_df

,Record_date,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name,Latitude,Longitude,Number of Records,Administrative District,East/West
67,2022-11-30,2022-11-27,12,2022-11-29,22,5100103,999,No,Bike,Theft,Aggravated theft,Spandau,Spandau Mitte / Nord,Hakenfelde,Mertensstraße,52.564493,13.216876,1.0,NaN,NaN
85,2022-11-30,2022-11-29,21,2022-11-30,7,5300839,599,No,Women,Theft,Aggravated theft,Spandau,Haselhorst / Siemensstadt,Siemensstadt,Rohrdamm,52.538629,13.263942,1.0,NaN,NaN
371,2022-11-24,2022-11-23,14,2022-11-23,21,5300736,2150,No,Women,Theft,Aggravated theft,Spandau,Haselhorst / Siemensstadt,Haselhorst,Zitadellenweg,52.539937,13.222603,1.0,NaN,NaN
376,2022-11-24,2022-11-24,15,2022-11-24,15,5200630,2500,No,Mountain,Theft,Theft,Spandau,Wilhelmstadt / Staaken,Wilhelmstadt,Wilhelmstadt West,52.515446,13.184676,1.0,NaN,NaN
420,2022-11-23,2022-11-20,16,2022-11-23,11,5200629,799,No,Men,Break-in,Aggravated theft from basement/floor,Spandau,Wilhelmstadt / Staaken,Wilhelmstadt,Borkumer Straße,52.532523,13.195180,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54713,2022-02-24,2022-02-23,10,2022-02-24,10,5100314,1303,No,diverse Fahrräder,Break-in,Aggravated theft from basement/floor,Spandau,Spandau Mitte / Nord,Spandau Mitte,Kurstraße,52.547406,13.207883,1.0,NaN,NaN
54718,2022-02-20,2022-02-20,6,2022-02-20,6,5200634,5510,No,diverse Fahrräder,Theft,Theft,Spandau,Wilhelmstadt / Staaken,Wilhelmstadt,Börnicker Straße,52.515631,13.188998,1.0,NaN,NaN
54719,2022-02-20,2022-02-18,17,2022-02-20,9,5200634,2000,No,diverse Fahrräder,Theft,Aggravated theft,Spandau,Wilhelmstadt / Staaken,Wilhelmstadt,Börnicker Straße,52.515631,13.188998,1.0,NaN,NaN
54754,2022-01-20,2022-01-19,22,2022-01-20,13,5200632,4771,No,diverse Fahrräder,Theft,Aggravated theft,Spandau,Wilhelmstadt / Staaken,Wilhelmstadt,Tiefwerder,52.514228,13.214642,1.0,NaN,NaN


In [106]:
spandau_df.drop(columns='Administrative District', inplace=True)

/var/folders/ry/gjr8200n3dlbt309r1yzb4480000gn/T/ipykernel_97151/440702370.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spandau_df.drop(columns='Administrative District', inplace=True)


In [108]:
spandau_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537 entries, 67 to 54756
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              1537 non-null   object 
 1   Stealing_date            1537 non-null   object 
 2   Stealing_hour            1537 non-null   int64  
 3   Report_stealing_date     1537 non-null   object 
 4   Report_stealing_hour     1537 non-null   int64  
 5   Berlin_code_area         1537 non-null   int64  
 6   Bike_value               1537 non-null   int64  
 7   Unsuccesful_attempt      1537 non-null   object 
 8   Bike_type                1537 non-null   object 
 9   Crime_type               1537 non-null   object 
 10  Crime_reason             1537 non-null   object 
 11  Administrative_district  1537 non-null   object 
 12  District                 1537 non-null   object 
 13  Quarter                  1537 non-null   object 
 14  Street_name           

In [109]:
spandau_df['East/West'] = 'West'

/var/folders/ry/gjr8200n3dlbt309r1yzb4480000gn/T/ipykernel_97151/3099724176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spandau_df['East/West'] = 'West'


In [113]:
spandau_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537 entries, 67 to 54756
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              1537 non-null   object 
 1   Stealing_date            1537 non-null   object 
 2   Stealing_hour            1537 non-null   int64  
 3   Report_stealing_date     1537 non-null   object 
 4   Report_stealing_hour     1537 non-null   int64  
 5   Berlin_code_area         1537 non-null   int64  
 6   Bike_value               1537 non-null   int64  
 7   Unsuccesful_attempt      1537 non-null   object 
 8   Bike_type                1537 non-null   object 
 9   Crime_type               1537 non-null   object 
 10  Crime_reason             1537 non-null   object 
 11  Administrative_district  1537 non-null   object 
 12  District                 1537 non-null   object 
 13  Quarter                  1537 non-null   object 
 14  Street_name           

In [112]:
new_column_names = {
    'Number of Records': 'Number_of_records',
    'East/West': 'East_west',
}
spandau_df.rename(columns=new_column_names, inplace=True)

/var/folders/ry/gjr8200n3dlbt309r1yzb4480000gn/T/ipykernel_97151/3277918953.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spandau_df.rename(columns=new_column_names, inplace=True)


In [122]:
df_west_east.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55004 entries, 0 to 55003
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              55004 non-null  object 
 1   Stealing_date            55004 non-null  object 
 2   Stealing_hour            55004 non-null  int64  
 3   Report_stealing_date     55004 non-null  object 
 4   Report_stealing_hour     55004 non-null  int64  
 5   Berlin_code_area         55004 non-null  int64  
 6   Bike_value               55004 non-null  int64  
 7   Unsuccesful_attempt      55004 non-null  object 
 8   Bike_type                55004 non-null  object 
 9   Crime_type               55004 non-null  object 
 10  Crime_reason             55004 non-null  object 
 11  Administrative_district  55004 non-null  object 
 12  District                 55004 non-null  object 
 13  Quarter                  55004 non-null  object 
 14  Street_name           

In [121]:
df_west_east.loc[df_west_east['Administrative_district'] == 'Tempelhof-Schöneberg', 'East/West'] = 'West'

In [119]:
east_west_filter = df_west_east[df_west_east['East/West'].isnull()]
east_west_filter 

,Record_date,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name,Latitude,Longitude,Number of Records,East/West
609,2022-11-20,2022-11-18,20,2022-11-20,13,7601546,450,No,Women,Theft,Aggravated theft,Tempelhof-Schöneberg,Marienfelde / Lichtenrade,Lichtenrade Süd,Franziusweg,43.111466,12.406088,1.0,NaN
638,2022-11-20,2022-11-18,3,2022-11-18,4,7601546,200,No,Bike,Theft,Aggravated theft,Tempelhof-Schöneberg,Marienfelde / Lichtenrade,Lichtenrade Süd,Franziusweg,43.111466,12.406088,1.0,NaN
829,2022-11-16,2022-11-15,12,2022-11-15,20,7601548,488,No,Men,Theft,Aggravated theft,Tempelhof-Schöneberg,Marienfelde / Lichtenrade,Lichtenrade Süd,Wittelsbacherstraße,52.384405,13.420599,1.0,NaN
843,2022-11-16,2022-11-14,7,2022-11-14,13,7601544,250,No,Mountain,Theft,Aggravated theft,Tempelhof-Schöneberg,Marienfelde / Lichtenrade,Lichtenrade Süd,John-Locke-Straße,52.390143,13.401957,1.0,NaN
859,2022-11-16,2022-11-15,22,2022-11-16,4,7601546,450,No,Men,Theft,Aggravated theft,Tempelhof-Schöneberg,Marienfelde / Lichtenrade,Lichtenrade Süd,Franziusweg,43.111466,12.406088,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53525,2022-12-12,2022-12-12,11,2022-12-12,19,7601238,5000,No,Men,Theft,Aggravated theft,Tempelhof-Schöneberg,Marienfelde / Lichtenrade,Marienfelde Nord,Marienfelde Nordost,52.472160,13.370287,1.0,NaN
53675,2022-12-08,2022-12-07,6,2022-12-07,17,7601238,311,No,Men,Theft,Aggravated theft,Tempelhof-Schöneberg,Marienfelde / Lichtenrade,Marienfelde Nord,Marienfelde Nordost,52.472160,13.370287,1.0,NaN
54176,2022-10-27,2022-09-09,18,2022-09-09,18,7601443,180,No,Women,Theft,Theft,Tempelhof-Schöneberg,Marienfelde / Lichtenrade,Lichtenrade Nord,Töpchiner Weg,52.406074,13.415051,1.0,NaN
54602,2022-05-02,2022-05-02,13,2022-05-02,13,7601546,1,No,diverse Fahrräder,Theft,Aggravated theft,Tempelhof-Schöneberg,Marienfelde / Lichtenrade,Lichtenrade Süd,Franziusweg,43.111466,12.406088,1.0,NaN


In [120]:
east_west_filter['Administrative_district'].value_counts()

Tempelhof-Schöneberg    340
Name: Administrative_district, dtype: int64

In [115]:
df_west_east.loc[df_west_east['Administrative_district'] == 'Spandau', 'East/West'] = 'West'

In [64]:
df_test = df_west_east.dropna()

In [65]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53127 entries, 0 to 55003
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              53127 non-null  object 
 1   Stealing_date            53127 non-null  object 
 2   Stealing_hour            53127 non-null  int64  
 3   Report_stealing_date     53127 non-null  object 
 4   Report_stealing_hour     53127 non-null  int64  
 5   Berlin_code_area         53127 non-null  int64  
 6   Bike_value               53127 non-null  int64  
 7   Unsuccesful_attempt      53127 non-null  object 
 8   Bike_type                53127 non-null  object 
 9   Crime_type               53127 non-null  object 
 10  Crime_reason             53127 non-null  object 
 11  Administrative_district  53127 non-null  object 
 12  District                 53127 non-null  object 
 13  Quarter                  53127 non-null  object 
 14  Street_name           

In [67]:
df_test.drop(columns='Administrative District', inplace=True)

/var/folders/ry/gjr8200n3dlbt309r1yzb4480000gn/T/ipykernel_97151/491157373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(columns='Administrative District', inplace=True)


In [124]:
df_west_east['Administrative_district'].value_counts()

Mitte                         8330
Pankow                        8029
Friedrichshain-Kreuzberg      7737
Charlottenburg-Wilmersdorf    5692
Tempelhof-Schöneberg          4737
Neukölln                      4178
Treptow-Köpenick              4176
Steglitz-Zehlendorf           3512
Lichtenberg                   3188
Reinickendorf                 2159
Marzahn-Hellersdorf           1729
Spandau                       1537
Name: Administrative_district, dtype: int64

In [125]:
new_column_names = {
    'Number of Records': 'Number_of_records',
    'East/West': 'East_west',
}
df_west_east.rename(columns=new_column_names, inplace=True)

In [126]:
df_west_east

,Record_date,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name,Latitude,Longitude,Number_of_records,East_west
0,2022-12-01,2022-12-01,8,2022-12-01,8,1300836,506,No,Bike,Theft,Aggravated theft,Mitte,Gesundbrunnen,Brunnenstraße Nord,Humboldthain Nordwest,52.540345,13.383037,1.0,East
1,2022-12-01,2022-11-29,22,2022-11-30,17,1300732,718,No,Bike,Theft,Aggravated theft,Mitte,Gesundbrunnen,Osloer Straße,Soldiner Straße,52.558950,13.385322,1.0,East
2,2022-12-01,2022-05-28,7,2022-05-28,14,6300526,569,No,Child,Theft,Aggravated theft,Steglitz-Zehlendorf,Zehlendorf Süd/Lichterfelde West,Teltower Damm,Zehlendorf Mitte,52.439345,13.259526,1.0,West
3,2022-12-01,2022-11-21,9,2022-11-21,16,1100309,600,No,Child,Theft,Aggravated theft,Mitte,Zentrum,Alexanderplatz,Oranienburger Straße,52.524646,13.394771,1.0,East
4,2022-12-01,2022-11-30,16,2022-12-01,11,3701659,400,No,Men,Theft,Aggravated theft,Pankow,Südlicher Prenzlauer Berg,Prenzlauer Berg Süd,Immanuelkirchstraße,52.532486,13.424236,1.0,East
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54999,2022-12-11 00:00:00,2022-12-09 00:00:00,21,2022-12-10 00:00:00,2,2300316,1200,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,1.0,West
55000,2022-12-08 00:00:00,2022-10-16 00:00:00,14,2022-10-16 00:00:00,16,4300412,200,No,Women,Theft,Aggravated theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Schloss Charlottenburg,Schlossgarten,52.537872,13.324342,1.0,West
55001,2022-11-29 00:00:00,2022-10-13 00:00:00,13,2022-10-13 00:00:00,22,4300412,400,No,Men,Theft,Theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Schloss Charlottenburg,Schlossgarten,52.537872,13.324342,1.0,West
55002,2022-10-26 00:00:00,2022-10-25 00:00:00,10,2022-10-25 00:00:00,15,2300316,479,No,Mountain,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,1.0,West


In [74]:
df_test.to_excel('tableau.xlsx',index=False)

In [75]:
df_test

,Record_date,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name,Latitude,Longitude,Number_of_records,East_west
0,2022-12-01,2022-12-01,8,2022-12-01,8,1300836,506,No,Bike,Theft,Aggravated theft,Mitte,Gesundbrunnen,Brunnenstraße Nord,Humboldthain Nordwest,52.540345,13.383037,1.0,East
1,2022-12-01,2022-11-29,22,2022-11-30,17,1300732,718,No,Bike,Theft,Aggravated theft,Mitte,Gesundbrunnen,Osloer Straße,Soldiner Straße,52.558950,13.385322,1.0,East
2,2022-12-01,2022-05-28,7,2022-05-28,14,6300526,569,No,Child,Theft,Aggravated theft,Steglitz-Zehlendorf,Zehlendorf Süd/Lichterfelde West,Teltower Damm,Zehlendorf Mitte,52.439345,13.259526,1.0,West
3,2022-12-01,2022-11-21,9,2022-11-21,16,1100309,600,No,Child,Theft,Aggravated theft,Mitte,Zentrum,Alexanderplatz,Oranienburger Straße,52.524646,13.394771,1.0,East
4,2022-12-01,2022-11-30,16,2022-12-01,11,3701659,400,No,Men,Theft,Aggravated theft,Pankow,Südlicher Prenzlauer Berg,Prenzlauer Berg Süd,Immanuelkirchstraße,52.532486,13.424236,1.0,East
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54999,2022-12-11 00:00:00,2022-12-09 00:00:00,21,2022-12-10 00:00:00,2,2300316,1200,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,1.0,West
55000,2022-12-08 00:00:00,2022-10-16 00:00:00,14,2022-10-16 00:00:00,16,4300412,200,No,Women,Theft,Aggravated theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Schloss Charlottenburg,Schlossgarten,52.537872,13.324342,1.0,West
55001,2022-11-29 00:00:00,2022-10-13 00:00:00,13,2022-10-13 00:00:00,22,4300412,400,No,Men,Theft,Theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Schloss Charlottenburg,Schlossgarten,52.537872,13.324342,1.0,West
55002,2022-10-26 00:00:00,2022-10-25 00:00:00,10,2022-10-25 00:00:00,15,2300316,479,No,Mountain,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Ost,Nördliche Luisenstadt,Köpenicker Straße,52.501895,13.441713,1.0,West


In [128]:
df_west_east.loc[df_west_east['Street_name'] == 'Reichenberger Straße Ost', 'East_west'] = 'East'


In [77]:
df_filter = df_test[df_test['Street_name'] == 'Reichenberger Straße Ost']


In [79]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 51 to 54748
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              328 non-null    object 
 1   Stealing_date            328 non-null    object 
 2   Stealing_hour            328 non-null    int64  
 3   Report_stealing_date     328 non-null    object 
 4   Report_stealing_hour     328 non-null    int64  
 5   Berlin_code_area         328 non-null    int64  
 6   Bike_value               328 non-null    int64  
 7   Unsuccesful_attempt      328 non-null    object 
 8   Bike_type                328 non-null    object 
 9   Crime_type               328 non-null    object 
 10  Crime_reason             328 non-null    object 
 11  Administrative_district  328 non-null    object 
 12  District                 328 non-null    object 
 13  Quarter                  328 non-null    object 
 14  Street_name            

In [98]:
df_test.to_excel('tableau.xlsx',index=False)

In [129]:
df_west_east.loc[df_west_east['Street_name'] == 'Reichenberger Straße West', 'Latitude'] = 52.496426777046665
df_west_east.loc[df_west_east['Street_name'] == 'Reichenberger Straße West', 'Longitude'] = 13.428167235020329

#52.496426777046665, 13.428167235020329

In [130]:
df_west_east.loc[df_west_east['Street_name'] == 'Uhlandstraße', 'East_west'] = 'West'

In [131]:
df_west_east['Bike_type'] = df_west_east['Bike_type'].replace({'diverse Fahrräder': 'Bike'})

In [132]:
df_west_east['Bike_type'].value_counts()

Men         25154
Women       15216
Bike         9183
Mountain     2343
Child        2168
Racing        819
Cargo         121
Name: Bike_type, dtype: int64

In [93]:
df_test['Bike_value'].value_counts().get(0,0)

381

In [96]:
zero_count = (df_test['Bike_value'] == 0).sum()
zero_count

381

In [133]:
df_west_east.loc[df_west_east['Street_name'] == 'Elsensee', 'Latitude'] = 52.501505660948176
df_west_east.loc[df_west_east['Street_name'] == 'Elsensee', 'Longitude'] = 13.603967036898574
# 52.501505660948176, 13.603967036898574

In [99]:
df_test['Administrative_district'].value_counts()

Mitte                         8330
Pankow                        8029
Friedrichshain-Kreuzberg      7737
Charlottenburg-Wilmersdorf    5692
Tempelhof-Schöneberg          4397
Neukölln                      4178
Treptow-Köpenick              4176
Steglitz-Zehlendorf           3512
Lichtenberg                   3188
Reinickendorf                 2159
Marzahn-Hellersdorf           1729
Name: Administrative_district, dtype: int64

In [111]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53127 entries, 0 to 55003
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Record_date              53127 non-null  object 
 1   Stealing_date            53127 non-null  object 
 2   Stealing_hour            53127 non-null  int64  
 3   Report_stealing_date     53127 non-null  object 
 4   Report_stealing_hour     53127 non-null  int64  
 5   Berlin_code_area         53127 non-null  int64  
 6   Bike_value               53127 non-null  int64  
 7   Unsuccesful_attempt      53127 non-null  object 
 8   Bike_type                53127 non-null  object 
 9   Crime_type               53127 non-null  object 
 10  Crime_reason             53127 non-null  object 
 11  Administrative_district  53127 non-null  object 
 12  District                 53127 non-null  object 
 13  Quarter                  53127 non-null  object 
 14  Street_name           

In [134]:
df_west_east.to_excel('tableau.xlsx',index=False)